In [ ]:
from obspy import read_inventory, UTCDateTime
from obspy.core.event import read_events
from obspy import read
import os
import subprocess
%matplotlib inline

"""
Double check that obspy is actually using an butterworth filter
Plot out some of the spectra for this data
"""

In [ ]:
path=os.getcwd()
EVID=385392
comparisionFrequency = 3.3

def write_st_as_sac(st,instrument="acc",eventID=0):
    directory = './'+str(eventID)+"/response"
    if not os.path.exists(directory):
        os.makedirs(directory)
    directory = directory + "/" + instrument + "/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    fname = directory+"/%s.sac" % trace.id
    trace.write(fname, format="SAC")

In [ ]:
### Clean up the input data
command,newFile = '',''

try:
    os.mkdir(path+"/"+str(EVID)+"/"+"cleaned")
    inputFiles = os.listdir(path+"/"+str(EVID))
    for file in inputFiles:
        net = "NN"
        sta,cha = file.split(".")[0],file.split(".")[1]
        newFile= net+'.'+sta+".."+cha
        command = "cp " + path+'/'+str(EVID)+'/'+file + ' ' + path+'/'+str(EVID)+'/'+"cleaned"+"/"+newFile + ".sac"
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        
except Exception as e:
    print(repr(e))
    print("you have most likely already cleaned up the input files")

In [ ]:
#begin by loading the station xml
inventory = read_inventory(path+"/reno.xml")

In [ ]:
#get all of the availible stations
st = read(path+"/385392/cleaned/*.sac")
#fix them so that they have the correct network metadata 
#(so that they can be found in the instrument response)
for i in st:
    i.meta["network"] = "NN"

In [ ]:
#Play with some filters and look at how well it works
"""
r1 = st[190].copy()
r2 = st[191].copy()
#filter down to target frequencty
r1Copy = r1.copy()
r1.filter('lowpass',freq=comparisionFrequency,corners=5,zerophase=True)
r1.plot()
r1Copy.plot()
r1.remove_response(inventory=inventory,output="acc",plot=True)
"""
def filterACC(trace,corners=True,zerophase=True,comparisionFrequency=comparisionFrequency):
    trace.filter('lowpass',freq=comparisionFrequency,corners=5,zerophase=True)
    pass

In [ ]:
#remove the response for all of the stations
#pre_filt = (0.05, 0.1, 50.0, 60.0)
plot = False
for trace in st:
    #attempt to remove the response for each
    try:
        #determine if accelerogram
        if("N" in trace.id.split("..")[1]):
            filterACC(trace)
            trace.remove_response(inventory=inventory,output="acc",plot=plot)
            #save to accelerogram dir
            write_st_as_sac(trace,instrument="acc",eventID=EVID)

        #velocity
        elif(("H" or "L") in trace.id.split("..")[1]):
            filterACC(trace)
            trace.remove_response(inventory=inventory,output="vel", pre_filt=pre_filt,plot=plot)            write_st_as_sac(trace,instrument="vel",eventID=EVID)
        #displacement
        elif("M" in trace.id.split("..")[1]):
            filterACC(trace)
            trace.remove_response(inventory=inventory,output="disp",plot=plot)
            write_st_as_sac(trace,instrument="disp",eventID=EVID)
        else:
            print("unkown seed code for " + str(trace))
        print("corrected response for " + str(trace))
    except Exception as e:
        #print(repr(e))
        pass